In [ ]:
!pyinstaller V5.py

In [23]:
import cv2
import numpy as np

# Загрузка изображения
image_path = './scr1.png'
image = cv2.imread(image_path)

# Проверка, успешно ли было загружено изображение
if image is None:
    print("Error loading image")
else:
    # Преобразование изображения из BGR в HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Вывод размеров изображения для понимания масштаба данных
    print(f"Dimensions of the image: {hsv_image.shape}")

    # Расчет и вывод минимальных и максимальных значений HSV по всему изображению
    min_hsv = np.min(hsv_image, axis=(0, 1))
    max_hsv = np.max(hsv_image, axis=(0, 1))

    print(f"Minimum HSV values: {min_hsv}")
    print(f"Maximum HSV values: {max_hsv}")

    # Отображение оригинального и HSV изображения
    cv2.imshow('Original Image', image)
    cv2.imshow('HSV Image', hsv_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Dimensions of the image: (98, 480, 3)
Minimum HSV values: [0 0 0]
Maximum HSV values: [178 255 190]


**Бинаризованное видео**

RGB

In [3]:
import cv2
import numpy as np
import time

# Открытие видеопотока
cap = cv2.VideoCapture("./SCHOM12.mp4")

# Глобальные переменные
drawing = False  # флаг рисования
roi = []  # область интереса
current_mouse_position = (0, 0)  # текущая позиция курсора
crossing_time = 0.6
buffer_duration = 0.4
last_contour_time = None
line_height = 100

threshold_r = 58
threshold_g = 62
threshold_b = 40

# Функция для обработки изменений положения ползунков
def on_threshold_r_change(value):
    global threshold_r
    threshold_r = value

def on_threshold_g_change(value):
    global threshold_g
    threshold_g = value

def on_threshold_b_change(value):
    global threshold_b
    threshold_b = value

# Функция для рисования прямоугольника
def draw_rectangle(event, x, y, flags, param):
    global roi, drawing, current_mouse_position

    current_mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi[2] = x
            roi[3] = y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        roi[2] = x
        roi[3] = y

# Функция для проверки валидности контура
def is_valid_contour(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    return aspect_ratio > 1

def detect_and_draw_contour(frame):
    global roi
    if roi and len(roi) == 4:
        x1, y1, x2, y2 = roi
        frame_roi = frame[y1:y2, x1:x2]
        if x1 < x2 and y1 < y2:
            frame_roi = frame[y1:y2, x1:x2]
            # Отрисовка прямоугольника вокруг ROI на исходном кадре
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        else:
            frame_roi = None
    else:
        frame_roi = frame
        

    if frame_roi is not None and frame_roi.size > 0:
    # Бинаризация по каналам RGB и комбинация результатов
        _, binary_r = cv2.threshold(frame_roi[:, :, 2], threshold_r, 255, cv2.THRESH_BINARY)    
        _, binary_g = cv2.threshold(frame_roi[:, :, 1], threshold_g, 255, cv2.THRESH_BINARY)    
        _, binary_b = cv2.threshold(frame_roi[:, :, 0], threshold_b, 255, cv2.THRESH_BINARY)    
        binary_combined = cv2.merge([binary_b, binary_g, binary_r]) 

        # Преобразование в градации серого
        gray = cv2.cvtColor(binary_combined, cv2.COLOR_BGR2GRAY)

        # Поиск контуров
        contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            valid_contours = [c for c in contours if cv2.contourArea(c) > 1 and is_valid_contour(c)]
            if valid_contours:
                max_contour = max(valid_contours, key=cv2.contourArea)
                cv2.drawContours(frame_roi, [max_contour], -1, (0, 255, 0), 3)
                check_contour_intersection(max_contour, frame_roi)

def check_contour_intersection(contour, frame):
    global line_height, contour_crossing_start, signal_start_time, signal_end_time, last_contour_time
    # Получаем координаты ограничивающего прямоугольника контура
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    # Вычисляем позицию линии относительно низа экрана
    line_position_from_bottom = frame.shape[0] - line_height
    # Проверяем, пересекает ли верхняя граница контура линию
    if y < line_position_from_bottom < y+h:
        last_contour_time = time.time()  # Обновляем время последнего обнаружения контура
        if contour_crossing_start is None:
            contour_crossing_start = time.time()  # Начало пересечения
        # Проверяем, находится ли контур в области интереса достаточно долго
        if time.time() - contour_crossing_start >= crossing_time:
            signal_start_time = True  # Сигнал активирован
            signal_end_time = time.time()  # Обновляем время окончания сигнала
    else:
        if last_contour_time and (time.time() - last_contour_time >= buffer_duration):
            # Деактивируем сигнал только если прошло достаточно времени после последнего обнаружения контура
            contour_crossing_start = None
            signal_start_time = False
            signal_end_time = None
            last_contour_time = None  # Сбрасываем время последнего обнаружения контура
        
    # Отображаем сигнал на экране, если он активен и длится достаточно долго
    if signal_start_time and signal_end_time:
        text = "Signal issued!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else: 
        text = "No Signal!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


def on_line_height_change(trackbarValue):
    global line_height, contour_crossing_start, signal_start_time
    line_height = trackbarValue
    contour_crossing_start = None  # Сброс времени начала пересечения
    signal_start_time = None  # Сброс времени начала сигнала

# Создание окна с ползунками
cv2.namedWindow("Frame")
cv2.createTrackbar('Threshold R', "Frame", threshold_r, 255, on_threshold_r_change)
cv2.createTrackbar('Threshold G', "Frame", threshold_g, 255, on_threshold_g_change)
cv2.createTrackbar('Threshold B', "Frame", threshold_b, 255, on_threshold_b_change)
cv2.setMouseCallback("Frame", draw_rectangle)

ret, frame = cap.read()

if ret:
    cv2.createTrackbar("Line Height", "Frame", line_height, frame.shape[0], on_line_height_change)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if roi:
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 0, 0), 2)

    detect_and_draw_contour(frame)

    _, binary_r = cv2.threshold(frame[:, :, 2], threshold_r, 255, cv2.THRESH_BINARY)    
    _, binary_g = cv2.threshold(frame[:, :, 1], threshold_g, 255, cv2.THRESH_BINARY)    
    _, binary_b = cv2.threshold(frame[:, :, 0], threshold_b, 255, cv2.THRESH_BINARY)    
    binary_combined = cv2.merge([binary_b, binary_g, binary_r]) 
    cv2.imshow("RGB", binary_combined)

    # Рисование линии с учетом ее положения от низа экрана
    cv2.line(frame, (0, frame.shape[0] - line_height), (frame.shape[1], frame.shape[0] - line_height), (255, 0, 0), 2)
    cv2.putText(frame, f"Cursor: {current_mouse_position}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    time.sleep(0.033)

    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break
    elif key == ord("r"):  # Сброс ROI
        roi = []

cap.release()
cv2.destroyAllWindows()

Binary

In [17]:
import cv2
import numpy as np
import time

# Открытие видеопотока
cap = cv2.VideoCapture("./SCHOM12.mp4")
threshold = 75

# Функция для обработки изменений положения ползунка
def on_threshold_change(value):
    global threshold
    threshold = value

# Создание окна с ползунком
cv2.namedWindow('Binary Video')
cv2.createTrackbar('Threshold', 'Binary Video', threshold, 255, on_threshold_change)

while True:
    # Чтение кадра из видеопотока
    ret, frame = cap.read()

    if not ret:
        break

    # Преобразование кадра в оттенки серого
    frame_blurred = cv2.GaussianBlur(frame, (3, 3), 0)
    gray = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2GRAY)
    # Динамическая корректировка порога бинаризации
    mean_intensity = np.mean(gray)
    dynamic_threshold = threshold * (0.8 + (mean_intensity - 128) / 128 * 0.95)
    # Бинаризация кадра
    _, binary = cv2.threshold(gray, dynamic_threshold, 255, cv2.THRESH_BINARY)
    
    # Морфологические операции
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Получение значения ползунка
    threshold = cv2.getTrackbarPos('Threshold', 'Binary Video')



    # Отображение бинаризованного кадра
    cv2.imshow('Binary Video', binary)
    time.sleep(0.03)


    # Выход из цикла при нажатии клавиши 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

HSV

In [4]:
import cv2
import numpy as np
import time

# Открытие видеопотока
cap = cv2.VideoCapture("./Try2.mp4")

# Начальные значения HSV порогов
lower_h, lower_s, lower_v = 0, 0, 40
upper_h, upper_s, upper_v = 180, 50, 220
# upper_h, upper_s, upper_v = 20, 255, 60 

# Глобальные переменные для функции draw_rectangle
drawing = False  # флаг рисования
roi = []  # область интереса
current_mouse_position = (0, 0)  # текущая позиция курсора

# Функции для обработки изменений положения ползунков
def on_lower_h_change(value):
    global lower_h
    lower_h = value

def on_lower_s_change(value):
    global lower_s
    lower_s = value

def on_lower_v_change(value):
    global lower_v
    lower_v = value

def on_upper_h_change(value):
    global upper_h
    upper_h = value

def on_upper_s_change(value):
    global upper_s
    upper_s = value

def on_upper_v_change(value):
    global upper_v
    upper_v = value

# Функция для рисования прямоугольника
def draw_rectangle(event, x, y, flags, param):
    global roi, drawing, current_mouse_position

    current_mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi[2] = x
            roi[3] = y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        roi[2] = x
        roi[3] = y

# Функция для проверки валидности контура
def is_valid_contour(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    # Предполагаем, что интересующие объекты будут иметь соотношение сторон ближе к горизонтальным
    return aspect_ratio > 0.5 



# Функция для обнаружения и отрисовки контура
def detect_and_draw_contour(frame):
    global roi
    if roi and len(roi) == 4:
        x1, y1, x2, y2 = roi
        frame_roi = frame[y1:y2, x1:x2]
        if x1 < x2 and y1 < y2:
            frame_roi = frame[y1:y2, x1:x2]
            # Отрисовка прямоугольника вокруг ROI на исходном кадре
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        else:
            frame_roi = None
    else:
        frame_roi = frame
        


    if frame_roi is not None and frame_roi.size > 0:
        #frame_blurred = cv2.GaussianBlur(frame_roi, (1, 1), 0)
        hsv = cv2.cvtColor(frame_roi, cv2.COLOR_BGR2HSV_FULL)
        mask = cv2.inRange(hsv, np.array([lower_h, lower_s, lower_v]), np.array([upper_h, upper_s, upper_v]))

        #kernel = np.ones((3, 3), np.uint8)
        #mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            valid_contours = [c for c in contours if cv2.contourArea(c) > 1 and is_valid_contour(c)]
            if valid_contours:
                max_contour = max(valid_contours, key=cv2.contourArea)
                cv2.drawContours(frame_roi, [max_contour], -1, (0, 255, 0), 3)


# Создание окна с ползунками и установка обработчика событий мыши
cv2.namedWindow('HSV Binary Video')
cv2.setMouseCallback('HSV Binary Video', draw_rectangle)

cv2.createTrackbar('Lower H', 'HSV Binary Video', lower_h, 179, on_lower_h_change)
cv2.createTrackbar('Lower S', 'HSV Binary Video', lower_s, 255, on_lower_s_change)
cv2.createTrackbar('Lower V', 'HSV Binary Video', lower_v, 255, on_lower_v_change)
cv2.createTrackbar('Upper H', 'HSV Binary Video', upper_h, 179, on_upper_h_change)
cv2.createTrackbar('Upper S', 'HSV Binary Video', upper_s, 255, on_upper_s_change)
cv2.createTrackbar('Upper V', 'HSV Binary Video', upper_v, 255, on_upper_v_change)

while True:
    # Чтение кадра из видеопотока
    ret, frame = cap.read()

    if not ret:
        break

    # Преобразование кадра в HSV и создание маски
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV_FULL)
    lower_bound = np.array([lower_h, lower_s, lower_v])
    upper_bound = np.array([upper_h, upper_s, upper_v])
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Отображение маски
    cv2.imshow('HSV Binary Video', mask)

    # Обнаружение и отрисовка контура
    detect_and_draw_contour(frame)

    # Отображение обработанного кадра
    cv2.imshow('Processed Frame', frame)

    time.sleep(0.05)

    # Выход из цикла при нажатии клавиши 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

**Песок**

****Новый****

In [32]:
import cv2
import time
import numpy as np

# Глобальные переменные
drawing = False  # флаг рисования
roi = []  # область интереса

line_height = 240  # начальная высота линии от низа экрана

# Новые переменные для порогов HSV
lower_hsv = np.array([0, 0, 40])  # Нижний порог HSV
upper_hsv = np.array([180, 50, 220])  # Верхний порог HSV

binary_threshold = 40 # порог бинаризации
current_mouse_position = (0, 0)  # текущая позиция курсора

contour_crossing_start = None  # Время начала пересечения контура с линией
signal_start_time = None  # Время начала сигнала
signal_end_time = None

crossing_time = 0.65
buffer_duration = 0.4 # Длительность буфера в секундах
last_contour_time = None  # Время последнего обнаружения контура в области интереса

def on_line_height_change(trackbarValue):
    global line_height, contour_crossing_start, signal_start_time
    line_height = trackbarValue
    contour_crossing_start = None  # Сброс времени начала пересечения
    signal_start_time = None  # Сброс времени начала сигнала
    

def draw_rectangle(event, x, y, flags, param):
    global roi, drawing, current_mouse_position

    current_mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi[2] = x
            roi[3] = y
    elif event == cv2.EVENT_LBUTTONUP:                                 
        drawing = False
        roi[2] = x          
        roi[3] = y


def detect_and_draw_contour(frame):
        # Проверяем, задана ли область интереса (ROI)
    if roi and len(roi) == 4:
        # Вырезаем ROI из кадра
        x1, y1, x2, y2 = roi
        frame_roi = frame[y1:y2, x1:x2]
        # Дополнительная проверка, чтобы убедиться, что координаты ROI корректны
        if x1 < x2 and y1 < y2:
            frame_roi = frame[y1:y2, x1:x2]
        else:
            frame_roi = None
    else:
        frame_roi = frame
    
    # Проверяем, не пуст ли frame_roi перед применением фильтра Гаусса
    if frame_roi is not None and frame_roi.size > 0:
    # Применяем фильтр Гаусса для сглаживания изображения
        frame_blurred = cv2.GaussianBlur(frame_roi, (5, 5), 0)
        hsv = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        #gray = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2HSV)
        # Динамическая корректировка порога бинаризации
        #mean_intensity = np.mean(gray)
        #dynamic_threshold = binary_threshold * (1 + (mean_intensity - 128) / 128 * 0.95)
        #_, binary = cv2.threshold(gray, dynamic_threshold, 255, cv2.THRESH_TOZERO_INV)
    
        # Морфологические операции
        #kernel = np.ones((7, 7), np.uint8)
        #mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            valid_contours = [c for c in contours if cv2.contourArea(c) > 1 and is_valid_contour(c)]
    
            if valid_contours:
                max_contour = max(valid_contours, key=cv2.contourArea)
                cv2.drawContours(frame_roi, [max_contour], -1, (0, 255, 0), 3)
                check_contour_intersection(max_contour, frame_roi)


def is_valid_contour(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    # Предполагаем, что кучка будет иметь соотношение сторон ближе к горизонтальным
    return aspect_ratio > 1

def check_contour_intersection(contour, frame):
    global line_height, contour_crossing_start, signal_start_time, signal_end_time, last_contour_time
    # Получаем координаты ограничивающего прямоугольника контура
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    # Вычисляем позицию линии относительно низа экрана
    line_position_from_bottom = frame.shape[0] - line_height
    # Проверяем, пересекает ли верхняя граница контура линию
    if y < line_position_from_bottom < y+h:
        last_contour_time = time.time()  # Обновляем время последнего обнаружения контура
        if contour_crossing_start is None:
            contour_crossing_start = time.time()  # Начало пересечения
        # Проверяем, находится ли контур в области интереса достаточно долго
        if time.time() - contour_crossing_start >= crossing_time:
            signal_start_time = True  # Сигнал активирован
            signal_end_time = time.time()  # Обновляем время окончания сигнала
    else:
        if last_contour_time and (time.time() - last_contour_time >= buffer_duration):
            # Деактивируем сигнал только если прошло достаточно времени после последнего обнаружения контура
            contour_crossing_start = None
            signal_start_time = False
            signal_end_time = None
            last_contour_time = None  # Сбрасываем время последнего обнаружения контура
        
    # Отображаем сигнал на экране, если он активен и длится достаточно долго
    if signal_start_time and signal_end_time:
        text = "Signal issued!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else: 
        text = "No Signal!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

cap = cv2.VideoCapture("./Try2.mp4")  # Используйте 0 для локальной каме

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", draw_rectangle)

# Получение размеров кадра для установки максимального значения ползунка
ret, frame = cap.read()
if ret:
    cv2.createTrackbar("Line Height", "Frame", line_height, frame.shape[0], on_line_height_change)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if roi:
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 0, 0), 2)

    detect_and_draw_contour(frame)

    # Рисование линии с учетом ее положения от низа экрана
    cv2.line(frame, (0, frame.shape[0] - line_height), (frame.shape[1], frame.shape[0] - line_height), (255, 0, 0), 2)
    cv2.putText(frame, f"Cursor: {current_mouse_position}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    time.sleep(0.03)

    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break
    elif key == ord("r"):  # Сброс ROI
        roi = []

cap.release()
cv2.destroyAllWindows()

**Програмные попыки**

In [1]:
import cv2
import time
import numpy as np

# Глобальные переменные
drawing = False  # флаг рисования
roi = []  # область интереса

line_height = 100  # начальная высота линии от низа экрана

binary_threshold = 40 # порог бинаризации
current_mouse_position = (0, 0)  # текущая позиция курсора

contour_crossing_start = None  # Время начала пересечения контура с линией
signal_start_time = None  # Время начала сигнала
signal_end_time = None

crossing_time = 0.65
buffer_duration = 0.4 # Длительность буфера в секундах
last_contour_time = None  # Время последнего обнаружения контура в области интереса

def on_line_height_change(trackbarValue):
    global line_height, contour_crossing_start, signal_start_time
    line_height = trackbarValue
    contour_crossing_start = None  # Сброс времени начала пересечения
    signal_start_time = None  # Сброс времени начала сигнала
    

def draw_rectangle(event, x, y, flags, param):
    global roi, drawing, current_mouse_position

    current_mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi[2] = x
            roi[3] = y
    elif event == cv2.EVENT_LBUTTONUP:                                 
        drawing = False
        roi[2] = x          
        roi[3] = y


def detect_and_draw_contour(frame):
        # Проверяем, задана ли область интереса (ROI)
    if roi and len(roi) == 4:
        # Вырезаем ROI из кадра
        x1, y1, x2, y2 = roi
        frame_roi = frame[y1:y2, x1:x2]
        # Дополнительная проверка, чтобы убедиться, что координаты ROI корректны
        if x1 < x2 and y1 < y2:
            frame_roi = frame[y1:y2, x1:x2]
        else:
            frame_roi = None
    else:
        frame_roi = frame
    
    # Проверяем, не пуст ли frame_roi перед применением фильтра Гаусса
    if frame_roi is not None and frame_roi.size > 0:
    # Применяем фильтр Гаусса для сглаживания изображения
        frame_blurred = cv2.GaussianBlur(frame_roi, (5, 5), 0)
    
        gray = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2GRAY)
        # Динамическая корректировка порога бинаризации
        mean_intensity = np.mean(gray)
        dynamic_threshold = binary_threshold * (1 + (mean_intensity - 128) / 128 * 0.95)
        _, binary = cv2.threshold(gray, dynamic_threshold, 255, cv2.THRESH_TOZERO_INV)
    
        # Морфологические операции
        kernel = np.ones((7, 7), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            valid_contours = [c for c in contours if cv2.contourArea(c) > 1 and is_valid_contour(c)]
    
            if valid_contours:
                max_contour = max(valid_contours, key=cv2.contourArea)
                cv2.drawContours(frame_roi, [max_contour], -1, (0, 255, 0), 3)
                check_contour_intersection(max_contour, frame_roi)


def is_valid_contour(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    # Предполагаем, что кучка будет иметь соотношение сторон ближе к горизонтальным
    return aspect_ratio > 1

def check_contour_intersection(contour, frame):
    global line_height, contour_crossing_start, signal_start_time, signal_end_time, last_contour_time
    # Получаем координаты ограничивающего прямоугольника контура
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    # Вычисляем позицию линии относительно низа экрана
    line_position_from_bottom = frame.shape[0] - line_height
    # Проверяем, пересекает ли верхняя граница контура линию
    if y < line_position_from_bottom < y+h:
        last_contour_time = time.time()  # Обновляем время последнего обнаружения контура
        if contour_crossing_start is None:
            contour_crossing_start = time.time()  # Начало пересечения
        # Проверяем, находится ли контур в области интереса достаточно долго
        if time.time() - contour_crossing_start >= crossing_time:
            signal_start_time = True  # Сигнал активирован
            signal_end_time = time.time()  # Обновляем время окончания сигнала
    else:
        if last_contour_time and (time.time() - last_contour_time >= buffer_duration):
            # Деактивируем сигнал только если прошло достаточно времени после последнего обнаружения контура
            contour_crossing_start = None
            signal_start_time = False
            signal_end_time = None
            last_contour_time = None  # Сбрасываем время последнего обнаружения контура
        
    # Отображаем сигнал на экране, если он активен и длится достаточно долго
    if signal_start_time and signal_end_time:
        text = "Signal issued!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else: 
        text = "No Signal!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

cap = cv2.VideoCapture("./SCHOM12.mp4")  # Используйте 0 для локальной камеры

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", draw_rectangle)

# Получение размеров кадра для установки максимального значения ползунка
ret, frame = cap.read()
if ret:
    cv2.createTrackbar("Line Height", "Frame", line_height, frame.shape[0], on_line_height_change)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if roi:
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 0, 0), 2)

    detect_and_draw_contour(frame)

    # Рисование линии с учетом ее положения от низа экрана
    cv2.line(frame, (0, frame.shape[0] - line_height), (frame.shape[1], frame.shape[0] - line_height), (255, 0, 0), 2)
    cv2.putText(frame, f"Cursor: {current_mouse_position}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    time.sleep(0.033)

    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break
    elif key == ord("r"):  # Сброс ROI
        roi = []

cap.release()
cv2.destroyAllWindows()

*ДЛЯ ДИПЛОМА*

In [ ]:
import cv2
import time
import numpy as np

# Глобальные переменные
drawing = False  # флаг рисования
roi = []  # область интереса

line_height = 240  # начальная высота линии от низа экрана

# Новые переменные для порогов HSV
lower_hsv = np.array([0, 0, 40])  # Нижний порог HSV
upper_hsv = np.array([180, 50, 220])  # Верхний порог HSV

binary_threshold = 40 # порог бинаризации
current_mouse_position = (0, 0)  # текущая позиция курсора

contour_crossing_start = None  # Время начала пересечения контура с линией
signal_start_time = None  # Время начала сигнала
signal_end_time = None # Подтвверждение окончания подачи сигнала

crossing_time = 0.65 # Требуемое время пересечения контуром границы для подачи сигнала
buffer_duration = 0.4 # Длительность буфера в секундах
last_contour_time = None  # Время последнего обнаружения контура в области интереса

# Функция для внесения изменений в границу высоты по средствам ползунка
def on_line_height_change(trackbarValue):
    global line_height, contour_crossing_start, signal_start_time
    line_height = trackbarValue
    contour_crossing_start = None  # Сброс времени начала пересечения
    signal_start_time = None  # Сброс времени начала сигнала
    
# Функция отрисовки зоны пользователя и получение координат для ограничения ROI
def draw_rectangle(event, x, y, flags, param):
    global roi, drawing, current_mouse_position

    current_mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi[2] = x
            roi[3] = y
    elif event == cv2.EVENT_LBUTTONUP:                                 
        drawing = False
        roi[2] = x          
        roi[3] = y


def detect_and_draw_contour(frame):
        # Проверяем, задана ли область интереса (ROI)
    if roi and len(roi) == 4:
        # Вырезаем ROI из кадра
        x1, y1, x2, y2 = roi
        frame_roi = frame[y1:y2, x1:x2]
        # Дополнительная проверка, чтобы убедиться, что координаты ROI корректны
        if x1 < x2 and y1 < y2:
            frame_roi = frame[y1:y2, x1:x2]
        else:
            frame_roi = None
    else:
        frame_roi = frame
    
    # Проверяем, не пуст ли frame_roi перед применением фильтра Гаусса
    if frame_roi is not None and frame_roi.size > 0:
    # Применяем фильтр Гаусса для сглаживания изображения
        frame_blurred = cv2.GaussianBlur(frame_roi, (5, 5), 0)
        hsv = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        # Поиск и сохранение контура под маской 
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Проверка контура на соответствие условиям
        if contours:
            valid_contours = [c for c in contours if cv2.contourArea(c) > 1 and is_valid_contour(c)]
            # Проверка на полноее соответствие и подача сигнала
            if valid_contours:
                max_contour = max(valid_contours, key=cv2.contourArea)
                cv2.drawContours(frame_roi, [max_contour], -1, (0, 255, 0), 3)
                check_contour_intersection(max_contour, frame_roi)


def is_valid_contour(contour):
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    # Предполагаем, что кучка будет иметь соотношение сторон ближе к горизонтальным
    return aspect_ratio > 1

def check_contour_intersection(contour, frame):
    global line_height, contour_crossing_start, signal_start_time, signal_end_time, last_contour_time
    # Получаем координаты ограничивающего прямоугольника контура
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    # Вычисляем позицию линии относительно низа экрана
    line_position_from_bottom = frame.shape[0] - line_height
    # Проверяем, пересекает ли верхняя граница контура линию
    if y < line_position_from_bottom < y+h:
        last_contour_time = time.time()  # Обновляем время последнего обнаружения контура
        if contour_crossing_start is None:
            contour_crossing_start = time.time()  # Начало пересечения
        # Проверяем, находится ли контур в области интереса достаточно долго
        if time.time() - contour_crossing_start >= crossing_time:
            signal_start_time = True  # Сигнал активирован
            signal_end_time = time.time()  # Обновляем время окончания сигнала
    else:
        if last_contour_time and (time.time() - last_contour_time >= buffer_duration):
            # Деактивируем сигнал только если прошло достаточно времени после последнего обнаружения контура
            contour_crossing_start = None
            signal_start_time = False
            signal_end_time = None
            last_contour_time = None  # Сбрасываем время последнего обнаружения контура
        
    # Отображаем сигнал на экране, если он активен и длится достаточно долго
    if signal_start_time and signal_end_time:
        text = "Signal issued!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else: 
        text = "No Signal!"
        cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

cap = cv2.VideoCapture("./Try2.mp4")  # Используйте 0 для локальной каме

# именования рабочего окна
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", draw_rectangle)

# Получение размеров кадра для установки максимального значения ползунка
ret, frame = cap.read()
if ret:
    cv2.createTrackbar("Line Height", "Frame", line_height, frame.shape[0], on_line_height_change)

while True:
    ret, frame = cap.read()
    #Проверка на правильность считывания первого кадра
    if not ret:
        break
    # отрисовка кадра
    if roi:
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 0, 0), 2)
    # Поск контура на кадре
    detect_and_draw_contour(frame)

    # Рисование линии с учетом ее положения от низа экрана
    cv2.line(frame, (0, frame.shape[0] - line_height), (frame.shape[1], frame.shape[0] - line_height), (255, 0, 0), 2)
    cv2.putText(frame, f"Cursor: {current_mouse_position}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Отрисовка кадра со всеми внесенными изменениями
    cv2.imshow("Frame", frame)
    time.sleep(0.03)

    # Возможность ручного завершения программы и сброса зоны выделенной пользователем
    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break
    elif key == ord("r"):  # Сброс ROI
        roi = []

cap.release()
cv2.destroyAllWindows()